### Load Formatter

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Load Modules

In [2]:
import pymc3 as pm
import numpy as np
from scipy import stats
import pandas as pd
from datetime import datetime
import plotly.express as px

<IPython.core.display.Javascript object>

### Generate some fake data

In [3]:
p_true = 0.75

pdf = pd.DataFrame()
for size in [10 ** i for i in range(1, 7)]:
    x = stats.bernoulli.rvs(p=p_true, size=size, random_state=42)
    pdf = pdf.append(pd.DataFrame({"size": size, "x": x}))

<IPython.core.display.Javascript object>

### Model with whole data

In [10]:
x = pdf.x.values[0:100]

with pm.Model() as model:
    p = pm.Beta("p", alpha=1, beta=1)
    obs = pm.Bernoulli("obs", p=p, observed=x)

    trace = pm.sample(1_000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 11 seconds.


<IPython.core.display.Javascript object>

In [40]:
def pandas_bernoulli_model(pdf: pd.DataFrame) -> pd.Series:

    t0 = datetime.now()

    x = pdf.x.values

    with pm.Model() as model:
        p = pm.Beta("p", alpha=1, beta=1)
        obs = pm.Bernoulli("obs", p=p, observed=x)
        trace = pm.sample(1_000, return_inferencedata=False)

    t1 = datetime.now()

    p_hat = trace["p"].mean()
    hdi = pm.hdi(trace["p"], hdi_prob=0.94)
    sample_time = trace.report.t_sampling
    deltat = t1 - t0

    return pd.Series(
        {
            "p_hat": p_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

<IPython.core.display.Javascript object>

In [42]:
results = pdf.groupby("size").apply(pandas_bernoulli_model)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
The acceptance probability does not match the target. It is 0.8804832072843767, but should be close to 0.8. Try to increase the number of tuning steps.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 11 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 16 seconds.
The acceptance probability does not match the target. It is 0.8810255019400122, but should be close to 0.8. Try to increase the number of tuning steps.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 80 seconds.


<IPython.core.display.Javascript object>

In [44]:
results.assign(runtime=lambda x: x.runtime.dt.total_seconds())

,p_hat,hdi_3%,hdi_97%,runtime,sample_time
size,,,,,
10,0.751227,0.533017,0.955473,11.847637,10.300048
100,0.756432,0.676535,0.832873,11.896031,10.352192
1000,0.756059,0.729839,0.781115,11.961634,10.307325
10000,0.758605,0.750400,0.766287,12.563323,10.755775
100000,0.750553,0.747921,0.753133,22.114658,16.402040
1000000,0.749893,0.749085,0.750729,216.195539,80.437720


<IPython.core.display.Javascript object>

In [64]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


def plot_results(results: pd.DataFrame, estimate: str, true_value: float):

    pdf_plot = results.reset_index().sort_values(["size"])

    x = pdf_plot["size"].values
    y = pdf_plot[estimate].values
    y_upper = pdf_plot["hdi_97%"].values
    y_lower = pdf_plot["hdi_3%"].values
    y2 = pdf_plot["sample_time"].values

    fig = (
        make_subplots(specs=[[{"secondary_y": True}]])
        .add_traces(
            [
                go.Scatter(
                    x=np.append(x, x[::-1]),  # x, then x reversed
                    y=np.append(y_upper, y_lower[::-1]),  # upper, then lower reversed
                    fill="toself",
                    fillcolor="rgba(0,100,80,0.2)",
                    line=dict(color="rgba(255,255,255,0)"),
                    #                 hoverinfo="skip",
                    name="HDI Bands (94%)",
                ),
                go.Scatter(
                    x=x,
                    y=y,
                    line=dict(color="rgb(0,100,80)"),
                    mode="lines",
                    name="Mean",
                ),
            ]
        )
        .add_trace(
            go.Scatter(x=x, y=y2, mode="lines", name="Runtime"), secondary_y=True
        )
        .add_hline(y=true_value, name="True Value", line_dash="dash")
        .update_xaxes(type="log", title="Data Size (N)")
        .update_yaxes(type="log", secondary_y=True, title="Seconds")
        .update_yaxes(title="Probability", secondary_y=False, type="log")
    )

    return fig

<IPython.core.display.Javascript object>

In [67]:
plot_results(results, "p_hat", p_true)

<IPython.core.display.Javascript object>

In [47]:
def weighted_bernoulli_loglike(p, weight, value):
    loglike = weight * pm.Bernoulli.dist(p=p).logp(value)
    return loglike


def pandas_bernoulli_model_agg(pdf: pd.DataFrame) -> pd.Series:

    t0 = datetime.now()

    pdf_agg = pdf.assign(freq=1).groupby(["x"]).agg({"freq": "count"}).reset_index()
    weights = pdf_agg.freq.values
    values = pdf_agg.x.values

    with pm.Model() as model:
        p = pm.Beta("p", alpha=1, beta=1)
        obs = pm.Potential(
            "obs", weighted_bernoulli_loglike(p=p, weight=weights, value=values)
        )
        trace = pm.sample(1_000, return_inferencedata=False)
    t1 = datetime.now()

    p_hat = trace["p"].mean()
    hdi = pm.hdi(trace["p"], hdi_prob=0.94)
    sample_time = trace.report.t_sampling
    deltat = t1 - t0

    return pd.Series(
        {
            "p_hat": p_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

<IPython.core.display.Javascript object>

In [48]:
results_agg = pdf.groupby("size").apply(pandas_bernoulli_model_agg)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 12 seconds.
The acceptance probability does not match the target. It is 0.8883089558104506, but should be close to 0.8. Try to increase the number of tuning steps.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
The acceptance probability does not match the target. It is 0.8843723247113201, but should be close to 0.8. Try to increase the number of tuning steps.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.


<IPython.core.display.Javascript object>

In [49]:
results_agg.assign(runtime=lambda x: x.runtime.dt.total_seconds())

,p_hat,hdi_3%,hdi_97%,runtime,sample_time
size,,,,,
10,0.751236,0.537472,0.954397,12.603707,11.556741
100,0.752583,0.673767,0.832658,11.125611,10.210250
1000,0.756531,0.732278,0.781910,11.365488,10.353999
10000,0.758663,0.750756,0.766811,11.152941,10.139856
100000,0.750546,0.748079,0.753195,11.106577,10.113874
1000000,0.749875,0.749066,0.750688,11.113571,10.091803


<IPython.core.display.Javascript object>

In [68]:
plot_results(results_agg, "p_hat", p_true)

<IPython.core.display.Javascript object>

### Poisson Example

In [73]:
mu_true = 100
pdf_poisson = pd.DataFrame()

for size in [10 ** j for j in range(1, 7)]:
    x = stats.poisson.rvs(mu_true, size=size, random_state=42)
    pdf_poisson = pdf_poisson.append(pd.DataFrame({"size": size, "x": x}))

<IPython.core.display.Javascript object>

In [74]:
def pandas_poisson_model(pdf: pd.DataFrame):

    t0 = datetime.now()

    x = pdf["x"].values

    with pm.Model() as model:
        mu = pm.Gamma("mu", alpha=1, beta=1)
        obs = pm.Poisson("obs", mu=mu, observed=x)
        trace = pm.sample(1_000, return_inferencedata=False)

    t1 = datetime.now()
    deltat = t1 - t0
    mu_hat = trace["mu"].mean()
    sample_time = trace.report.t_sampling
    hdi = pm.hdi(trace["mu"], hdi_prob=0.94)

    return pd.Series(
        {
            "mu_hat": mu_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

<IPython.core.display.Javascript object>

In [ ]:
poisson_results = pdf_poisson.groupby(["size"]).apply(pandas_poisson_model)

DEBUG:filelock:Attempting to acquire lock 6626666336 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6626666336 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6626666336 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6626666336 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6626837408 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6626837408 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6626837408 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEB

DEBUG:filelock:Lock 6529346240 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6529346240 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6529346240 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6502951952 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6502951952 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6502951952 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6502951952 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
Multiprocess sam

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
INFO:pymc3:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
The acceptance probability does not match the target. It is 0.8801657829349819, but should be close to 0.8. Try to increase the number of tuning steps.
DEBUG:filelock:Attempting to acquire lock 6533280576 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6533280576 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6533280576 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6533280576 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6523139792 on /Users/

DEBUG:filelock:Lock 6500719008 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6500719008 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6500719008 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6592824656 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6592824656 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6592824656 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6592824656 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:A

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
INFO:pymc3:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
DEBUG:filelock:Attempting to acquire lock 6624657856 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6624657856 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6624657856 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6624657856 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6625852576 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6625852576 acquired on /Users/felipepinochet/.t

...

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 11 seconds.
INFO:pymc3:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 11 seconds.
DEBUG:filelock:Attempting to acquire lock 6627969680 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6627969680 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6627969680 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6627969680 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6622678800 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6622678800 acquired on /Users/felipepinochet/.t

Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
DEBUG:filelock:Attempting to acquire lock 6625841056 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6625841056 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6625841056 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6625841056 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6626193520 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6626193520 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 66261935

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 19 seconds.
INFO:pymc3:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 19 seconds.
DEBUG:filelock:Attempting to acquire lock 6539240976 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6539240976 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6539240976 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6539240976 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6555324768 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6555324768 acquired on /Users/felipepinochet/.t

Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
DEBUG:filelock:Attempting to acquire lock 6529195696 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6529195696 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 6529195696 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6529195696 released on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to acquire lock 6588942896 on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Lock 6588942896 acquired on /Users/felipepinochet/.theano/compiledir_macOS-11.2.3-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG:filelock:Attempting to release lock 65889428

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 162 seconds.
INFO:pymc3:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 162 seconds.


In [ ]:
poisson_results

In [ ]:
def poisson_likelihood(weight, value, mu):
    return weight * pm.Poisson.dist(mu=mu).logp(value)


def pandas_poisson_model_agg(pdf: pd.DataFrame):

    t0 = datetime.now()

    pdf_agg = pdf.assign(freq=1).groupby(["x"]).agg({"freq": "count"}).reset_index()

    values = pdf_agg["x"].values
    weights = pdf_agg["freq"].values

    with pm.Model() as model:
        mu = pm.Gamma("mu", alpha=1, beta=1)
        obs = pm.Potential(
            "obs", poisson_likelihood(weight=weights, value=values, mu=mu)
        )
        trace = pm.sample(1_000, return_inferencedata=False)

    t1 = datetime.now()
    deltat = t1 - t0
    mu_hat = trace["mu"].mean()
    sample_time = trace.report.t_sampling
    hdi = pm.hdi(trace["mu"], hdi_prob=0.94)

    return pd.Series(
        {
            "mu_hat": mu_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

In [ ]:
poisson_results_agg = pdf_poisson.groupby(["size"]).apply(pandas_poisson_model_agg)

In [ ]:
poisson_results_agg

In [ ]:
plot_results(poisson_results, "mu_hat", mu_true).update_traces(mode="lines+markers")

In [ ]:
plot_results(poisson_results_agg, "mu_hat", mu_true).update_traces(mode="lines+markers")